# Dog Breed Identification

Using image classification models to identify dog breeds from pictures of man's best friend.

## Project Goal:

This project aims to create a model that can take in the image of a dog and correctly assess that animal's breed. This is being made as a tool that could potentially be used by pounds / animal shelters to assist them in properly labeling the dogs that they take in. 

## Aims:

This project intends to:

 - Take in ~20.6k images of 120 different dog breeds and create a convolutional neural network (see: Definitions) that can identify dogs of those 120 breeds. 
 - If unable to use all 120 breeds, just use as many as possible and use the final model as a proof of concept. 

## Definitions: 

- Convolutional neural network (CNN): A form of deep neural netwrok model commonly used to analyze images. You can read more about them [here.](https://en.wikipedia.org/wiki/Convolutional_neural_network)
- Epoch: One full cycle through a training dataset. 
- Class: A set of data that represents something, in this case a collection of images of a dog breed. 

## Data:

This project utilizes images taken from the [Stanford Dogs Dataset on kaggle](https://www.kaggle.com/jessicali9530/stanford-dogs-dataset), provided by Stanford University. 

The dataset contains 20,580 images of 120 different breeds of dogs. Each dog breed has at the minimum 150 images to represent them. Each breed has its own image folder which can be used as a class in a CNN. The breeds used across all the models created were the Bluetick, Boxer, German Shepherd, Golden Retreiver, Pembroke (Welsh Corgi), Pug, Pomeranian, Saluki, Maltese Dog, and Afghan Hound. 

The only breeds not chosen randomly were the Maltese Dog and Afghan Hound. They were chosen to be used on the final model as they have the most images out of all the other breeds (252 and 239 images respectively). 

The hyperlink above leads to the kaggle page but if you wish to head to the original data source you may head [here.](http://vision.stanford.edu/aditya86/ImageNetDogs/)


While working on this dataset I had used 3 different train-test splits. Being the percent train, test, and validation data respectively, they were first 90 9 1, then 70 15 15, then the data was split up 90 5 5. The reason for the first split being replaced was because the very first models only had 1% of the images to use for validation data. After a couple iterations I had noticed that the validation accuracy frequently came up as 'clean' numbers such as .5 or .75. 15% valdiation data worked fine but then the testing accuracy was too low. This led to the decision to give the training data more images to work with to hopefuly improve the models, which is how I ended up with the 90 5 5 split. 

## EDA:

The only EDA I had done was create a histogram of the distribution of the image counts of the dog breed folders to visualize how uneven the image representation was among the breeds. 

![Dist_plot](../Vis_Files/img_dist.png)

## Models:

This project only uses CNN's for its image classifciation. 

The final model was evaluated based on its test accuracy score. The first simple model only used its training accuarcy score. All models beyond the first simple model while in the iterative process had used the validation accuracy to determine their effectivness. 

Accuracy was the metric used to evaluate the models because there are little to no serious consequences for incorrectly guessing a dogs breed as another. 

### First Simple Model

The first simple model had used 3 breeds, the bluetick, boxer, and great dane. It was set up like this:

In [ ]:
train_generator = train_datagen.flow_from_directory(
        ExModeling_train,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')
fsm_train_generator.class_indices

fsm_model = models.Sequential()
fsm_model.add(layers.Dense(64, input_shape = (150, 150, 3)))
fsm_model.add(layers.Flatten())
fsm_model.add(layers.Dense(3, activation = 'softmax'))
fsm_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = fsm_model.fit_generator(train_generator, steps_per_epoch=100, epochs=5)


This neted a training accuracy score of 0.359, which is slightly better than randomly guessing with 3 classes. I had not used any validation data yet, but based on the very low accuracy score it was clear that there were going to need to be some changes. 

The next major step I took was adding several convolution layers and checking validation accuracy every epoch. It should be noted that the dog breeds were shuffled up at this point, so the breeds being used are the golden retriever, pembroke (corgi), pug, pomeranian, and saluki. The reason I had used more breeds than before was because I believed that I would be able to reach all 120 classes/ breeds. Going from 3 to 5 did not seem like a large leap with that in mind. This led to a model that looked something like this:

In [ ]:
model_3 = models.Sequential()
model_3.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))#imput layer
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_3.add(layers.MaxPooling2D((2, 2)))
model_3.add(layers.Flatten())
model_3.add(layers.Dense(64, activation='relu'))
model_3.add(layers.Dense(128, activation='relu'))
model_3.add(layers.Dense(256, activation='relu'))
model_3.add(layers.Dense(5, activation='softmax'))

model_3.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

history_3 = model_3.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=17, 
    validation_data=validation_generator,
    validation_steps=50)

![bad_model.png](../Vis_Files/bad_model.png)

This model resulted in a higher training accuracy score of .587 but the validation accuracy was only 0.417. Clearly the model was overfitting. 

The way I successfully managed to combat this overfitting was by using regularization and using a greater % of the data as training data (from 70% of all images to 90%). The model looked like this:

In [ ]:
model_e = models.Sequential()
model_e.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model_e.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_e.add(layers.MaxPooling2D((2, 2)))
model_e.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_e.add(layers.MaxPooling2D((2, 2)))
model_e.add(layers.Conv2D(128, (3, 3)))
model_e.add(layers.MaxPooling2D((2, 2)))
model_e.add(layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01) ))
model_e.add(layers.MaxPooling2D((2, 2)))
model_e.add(layers.Flatten())
model_e.add(layers.Dense(64, activation='relu'))
model_e.add(layers.Dense(64, activation='relu'))
model_e.add(layers.Dense(128, activation='relu',))
model_e.add(layers.Dense(5, activation='softmax'))

model_e.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

history_e = model_e.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=15, #more 
    validation_data=validation_generator,
    validation_steps=50)

![model_2_plot](../Vis_Files/model_2.png)

The training accuracy was .383 and the validation accuracy was .374. Although these numbers were low, they were similar, which means I was no longer overfitting. 

At this point my models were taking a long time to run and with 5 classes and 15 epochs the results were not great. In order to progress further I would need more complex models that would need to train on more epochs, and my computer likely could not handle much more than this. 
 

### Final Model

I decided to cut my losses and see how well the same model would perform with just 2 breeds/ classes. The breeds chosen were the maltese dog and afghan hound. They were chosen because they had the most images out of all the other breeds. The resulting model, which would be the final model, looked like this:

In [ ]:
model_1 = models.Sequential() 
model_1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model_1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(128, (3, 3)))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01) ))
model_1.add(layers.MaxPooling2D((2, 2)))
model_1.add(layers.Flatten())
model_1.add(layers.Dense(64, activation='relu'))
model_1.add(layers.Dense(64, activation='relu'))
model_1.add(layers.Dense(128, activation='relu',))
model_1.add(layers.Dense(2, activation='softmax'))

model_1.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

history_1 = model_1.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=50)

![model_3_plot](../Vis_Files/Final_acc.png)

## Results and Next Steps

### Best Model

The final and best performing model was model_1 from the 'Modeling_2' notebook with a train accuracy of .850 and a test accuracy of 0.854. Overfitting was a big problem early on in the modeling process so I am glad to see evidence of overcoming it. This certainly isn't enough to handle all 120 breeds, but it is a good start. 

### Future Improvments

- A limitation of this project was processing power. My models could only get so complex or else they would take too long to run and it was evident that the internal temperatures of my mac book were getting high while running them. 
- Once processing power is no longer a problem, the next step would be to run more advanced models over more epochs. Most models run in this project only had 10 or 15 epochs. 
- One improvment that could only make the project better would be to add more dog images beyond those provided by the Stanford dataset. Many dog breeds only had 150 images representing them, and more would have been prefered. 

## Sources

https://github.com/learn-co-students/dsc-image-classification-lab-sea01-dtsc-ft-051120/tree/solution

https://www.kaggle.com/jessicali9530/stanford-dogs-dataset

Aditya Khosla, Nityananda Jayadevaprakash, Bangpeng Yao and Li Fei-Fei. Novel dataset for Fine-Grained Image Categorization. First Workshop on Fine-Grained Visual Categorization (FGVC), IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2011.

J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li and L. Fei-Fei, ImageNet: A Large-Scale Hierarchical Image Database. IEEE Computer Vision and Pattern Recognition (CVPR), 2009.